# Modelクラスのコードリーディング

## setup

import

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import threading

Contextクラス

In [3]:
class Context(object):
    """Functionality for objects that put themselves in a context using
    the `with` statement.
    """
    contexts = threading.local()

    def __enter__(self):
        type(self).get_contexts().append(self)
        return self

    def __exit__(self, typ, value, traceback):
        type(self).get_contexts().pop()

# Modelクラス

In [6]:
class Model(Context):
    def __new__(cls, *args, **kwargs):
        instance = super(Model, cls).__new__(cls)
        if kwargs.get('model') is not None:
            instance.parent = kwargs.get('model')
        elif cls.get_contexts():
            instance.parent = cls.get_contexts()[-1]
        else:
            instance.parent = None
            
    
    def __init__(self, name="", model=None, ):
        self.name = name
        if self.parent is not None:
            self.named_vars = treedict(parent=self.parent.named_vars)
        else:
            self.named_vars = treedict()
            
    @property
    def model(self):
        return self
    
    @property
    def decription(self):
        return
    
    @classmethod
    def get_contexts(cls):
        # no race-condition here, cls.contexts is a thread-local object
        # be sure not to override contexts in a subclass however!
        if not hasattr(cls.contexts, 'stack'):
            cls.contexts.stack = []
        return cls.contexts.stack

    @classmethod
    def get_context(cls):
        """Return the deepest context on the stack."""
        try:
            return cls.get_contexts()[-1]
        except IndexError:
            raise TypeError("No context on context stack")
            
    def add_random_variable(self, var):
        """Add a random variable to the named variables of the model."""
        if self.named_vars.tree_contains(var.name):
            raise ValueError(
                "Variable name {} already exists.".format(var.name))
        self.named_vars[var.name] = var

TODO there may be many things I can play with "Model" class